In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [45]:
batch = 40
cuda_use = torch.cuda.is_available()
epochs = 20
lr = 0.001
momentum = 0.9
seed = 1

In [46]:
if cuda_use:
    torch.manual_seed(seed)
train_loader = torch.utils.data.DataLoader(
datasets.MNIST('./MNIST_data', train=True, download=True,
               transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ])),batch_size = batch, shuffle = True,num_workers = 2)

test_loader = torch.utils.data.DataLoader(
datasets.MNIST('./MNIST_data', train=False, transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ])),batch_size = batch, shuffle = True,num_workers = 2)

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=(5,5),stride=1, padding=0)
        self.conv3 = nn.Conv2d(6, 6, kernel_size=(3,3),stride=1, padding=1)
        self.conv3 = nn.Conv2d(6, 6, kernel_size=(3,3),stride=1, padding=1)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=(5,5),stride=1, padding=0)
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv3(out))
        out = F.relu(self.conv3(out))
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1) #flatten
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out
model = Net()
if cuda_use:
    device = torch.device('cuda:0')
    model.to(device)
Loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [51]:
def adjust_learning_rate(optimizer, epoch):
    if epoch < 10:
        lr = 0.01
    elif epoch < 15:
        lr = 0.001
    else: 
        lr = 0.0001

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
def train(epoch):
    model.train()
    adjust_learning_rate(optimizer, epoch)

    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda_use:
            data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = Loss(output, target)
        loss.backward()
        optimizer.step()
#         print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.data))

#Testing function
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if cuda_use:
            data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        test_loss += Loss(output, target).data
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

#run and save model
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    savefilename = 'LeNet_'+str(epoch)+'.tar'
#     torch.save({
#             'epoch': epoch,
#             'state_dict': model.state_dict(),
#         }, savefilename)


Test set: Average loss: 0.0692, Accuracy: 9786/10000 (97%)


Test set: Average loss: 0.0413, Accuracy: 9868/10000 (98%)


Test set: Average loss: 0.0515, Accuracy: 9841/10000 (98%)


Test set: Average loss: 0.0427, Accuracy: 9867/10000 (98%)


Test set: Average loss: 0.0438, Accuracy: 9871/10000 (98%)


Test set: Average loss: 0.0414, Accuracy: 9878/10000 (98%)


Test set: Average loss: 0.0545, Accuracy: 9854/10000 (98%)


Test set: Average loss: 0.0532, Accuracy: 9846/10000 (98%)

